# Створення нейронної мережі

У цьому завданні ми створимо повнозв'язну нейронну мережу, використовуючи при цьому низькорівневі механізми tensorflow.

Архітектура нейромережі представлена на наступному малюнку. Як бачиш, у ній є один вхідний шар, два приховані, а також вихідний шар. В якості активаційної функції у прихованих шарах буде використовуватись сигмоїда. На вихідному шарі ми використовуємо softmax.

Частина коду зі створення мережі вже написана, тобі потрібно заповнити пропуски у вказаних місцях.

## Архітектура нейронної мережі

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>


## Про датасет MNIST

Дану нейромережу ми будемо вивчати на датасеті MNIST. Цей датасет являє собою велику кількість зображень рукописних цифр розміром $28 \times 28$ пікселів. Кожен піксель приймає значення від 0 до 255.

Як і раніше, датасет буде розділений на навчальну та тестову вибірки. При цьому ми виконаємо нормалізацію всіх зображень, щоб значення пікселів знаходилось у проміжку від 0 до 1, розділивши яскравість кожного пікселя на 255.

Окрім того, архітектура нейронної мережі очікує на вхід вектор. У нашому ж випадку кожен об'єкт вибірки являє собою матрицю. Що ж робити? У цьому завданні ми "розтягнемо" матрицю $28 \times 28$, отримавши при цьому вектор, що складається з 784 елементів.

![MNIST Dataset](https://www.researchgate.net/profile/Steven-Young-5/publication/306056875/figure/fig1/AS:393921575309346@1470929630835/Example-images-from-the-MNIST-dataset.png)

Більше інформації про датасет можна знайти [тут](http://yann.lecun.com/exdb/mnist/).

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import keras as K
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [22]:
num_classes = 10 # загальна кількість класів, у нашому випадку це цифри від 0 до 9
num_features = 784 # кількість атрибутів вхідного вектора 28 * 28 = 784

learning_rate = 0.001 # швидкість навчання нейронної мережі
training_steps = 3000 # максимальне число епох
batch_size = 256 # перераховувати ваги мережі ми будемо не на всій вибірці, а на її випадковій підмножині з batch_size елементів
display_step = 100 # кожні 100 ітерацій ми будемо показувати поточне значення функції втрат і точності

n_hidden_1 = 128 # кількість нейронів 1-го шару
n_hidden_2 = 256 # кількість нейронів 2-го шару

In [3]:
# from tensorflow.keras.datasets import mnist
from keras.datasets import mnist

# Завантажуємо датасет
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Перетворюємо цілочисельні пікселі на тип float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Перетворюємо матриці розміром 28x28 пікселів у вектор з 784 елементів
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Нормалізуємо значення пікселів
x_train, x_test = x_train / 255., x_test / 255.

# Перемішаємо тренувальні дані
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [4]:
# Створимо нейронну мережу

class DenseLayer(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features]), name="w"
        )
        self.b = tf.Variable(tf.zeros([out_features]), name="b")

    def __call__(self, x, activation=0):
        y = tf.matmul(x, self.w) + self.b
        if activation != 0:
            return tf.nn.softmax(y)
        else:
            return tf.nn.sigmoid(y)

class NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    # Перший шар, який складається з 128 нейронів
    # Місце для вашого коду
    self.layer_1 = DenseLayer(in_features=num_features, out_features=n_hidden_1, name="Layer1")
    # Другий шар, який складається з 256 нейронів
    # Місце для вашого коду
    self.layer_2 = DenseLayer(in_features=n_hidden_1, out_features=n_hidden_2, name="Layer2")
    # Вихідний шар
    # Місце для вашого коду
    self.layer_3 = DenseLayer(in_features=n_hidden_2, out_features=num_classes, name="Output")


  def __call__(self, x):
    # Передача даних через перші два шари та вихідний шар з функцією активації softmax
    # Місце для вашого коду
    x = self.layer_1(x)
    x = self.layer_2(x)

    return self.layer_3(x, activation=tf.nn.softmax) # Місце для вашого коду

In [17]:
# В якості функції помилки в даному випадку зручно взяти крос-ентропію
def cross_entropy(y_pred, y_true):
    # Закодувати label в one hot vector
    y_true = tf.one_hot(y_true, depth=num_classes)

    # Значення передбачення, щоб уникнути помилки log(0).
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Обчислення крос-ентропії
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Як метрику якості використовуємо точність
def accuracy(y_pred, y_true):
    # Місце для вашого коду
    y_pred_labels = tf.cast(tf.argmax(y_pred, axis=1), tf.float64)

    correct_predict = tf.equal(y_pred_labels, tf.cast(y_true, tf.float64))

    accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))

    return accuracy

In [20]:
# Створимо екземпляр нейронної мережі
neural_net = NN(name="mnist")

# Функція навчання нейромережі
def train(neural_net, input_x, output_y):
  # Для налаштування вагів мережі будемо використовувати стохастичний градієнтний спуск
  optimizer = tf.optimizers.SGD(learning_rate)

  # Активація автоматичного диференціювання
  with tf.GradientTape() as g:
    pred = neural_net(input_x)
    loss = cross_entropy(pred, output_y)

    # Отримаємо список оптимізованих параметрів
    # Місце для вашого коду
    gradients = g.gradient(loss, neural_net.trainable_variables)
    # Обчислимо за ними значення градієнта
    # Місце для вашого коду
    optimizer.apply_gradients(zip(gradients, neural_net.trainable_variables))

    # Модифікуємо параметри
    # Місце для вашого коду
    # optimizer.minimize(loss, neural_net.trainable_variables)

In [23]:
# Тренування мережі

loss_history = []  # кожні display_step кроків зберігай в цьому списку поточну помилку нейромережі
accuracy_history = [] # кожні display_step кроків зберігай в цьому списку поточну точність нейромережі

# У цьому циклі ми будемо проводити навчання нейронної мережі
# із тренувального датасета train_data вилучи випадкову підмножину, на якій
# відбудеться тренування. Використовуй метод take, доступний для тренувального датасета.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Оновлюємо ваги нейронної мережі
    # Місце для вашого коду
    train(neural_net, batch_x, batch_y)

    if step % display_step == 0:
        pred = neural_net(batch_x)
        # Місце для вашого коду
        loss_step = cross_entropy(pred, batch_y)
        accuracy_step = accuracy(pred, batch_y)

        loss_history.append(loss_step)
        accuracy_history.append(accuracy_step)

        print(f"Step: {step}, Loss: {loss_step:7.2f}, Accuracy: {accuracy_step:.2%}")


Step: 100, Loss:   52.43, Accuracy: 94.92%
Step: 200, Loss:   28.73, Accuracy: 96.09%
Step: 300, Loss:   54.30, Accuracy: 92.58%
Step: 400, Loss:   61.21, Accuracy: 91.41%
Step: 500, Loss:   35.37, Accuracy: 95.70%
Step: 600, Loss:   51.12, Accuracy: 94.14%
Step: 700, Loss:   25.44, Accuracy: 96.88%
Step: 800, Loss:   41.69, Accuracy: 95.70%
Step: 900, Loss:   29.26, Accuracy: 96.88%
Step: 1000, Loss:   34.55, Accuracy: 96.88%
Step: 1100, Loss:   23.64, Accuracy: 96.48%
Step: 1200, Loss:   19.66, Accuracy: 97.66%
Step: 1300, Loss:   33.70, Accuracy: 97.27%
Step: 1400, Loss:   22.52, Accuracy: 97.66%
Step: 1500, Loss:   41.22, Accuracy: 95.70%
Step: 1600, Loss:   24.44, Accuracy: 97.27%
Step: 1700, Loss:   30.26, Accuracy: 96.48%
Step: 1800, Loss:   38.38, Accuracy: 96.88%
Step: 1900, Loss:   31.62, Accuracy: 96.48%
Step: 2000, Loss:   26.99, Accuracy: 96.48%
Step: 2100, Loss:   27.73, Accuracy: 97.27%
Step: 2200, Loss:   39.92, Accuracy: 95.70%
Step: 2300, Loss:   36.08, Accuracy: 94.5

In [ ]:
# Виведіть графіки залежності зміни точності і втрат від кроку
# Якщо все зроблено правильно, то точність повинна зростати, а втрати зменшуватись

import matplotlib.pyplot as plt

# Виведіть графік функції втрат
# Місце для вашого коду

# Виведіть графік точності
# Місце для вашого коду


In [ ]:
# Обчисліть точність навченої нейромережі
# Місце для вашого коду
# Тестування моделі на тестових даних
# Місце для вашого коду

In [ ]:
# Протестуйте навчену нейромережу на 10 зображеннях. З тестової вибірки візьміть 5
# випадкових зображень і передайте їх у нейронну мережу.
# Виведіть зображення та випишіть  поруч відповідь нейромережі.
# Зробіть висновок про те, чи помиляється твоя нейронна мережа, і якщо так, то як часто?

# Місце для вашого коду


